In [1]:
#import libraries
import os
import json
import numpy as np
import cv2 as cv
import random

In [10]:
#define paths

#here are the original train anotations in json
path_labels    = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\train_annotations'   

#path where, the are the decoded train images. here is necessary to have images in their full resolution, 
#without top/bottom crop
path_images    = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\images' 

#path where, the are the decoded train images
out_dir        = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher'

path_out_file  = r'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_matcher2.txt'

In [3]:
#array that defines which classes have to be extracted. 
#classes = ['Car', 'Pedestrian', 'Truck', 'Signal', 'Signs', 'Bicycle', 'Motorbike', 'Bus', 'Svehicle', 'Train']
classes = ['Car', 'Pedestrian']

In [4]:
#some supporting functions
def normalize_image_size(image, target_width, target_height, default_background_color=0, refiner_on=False):
    height, width, _ = image.shape
    height_ratio = height / target_height
    width_ratio = width / target_width
    if width_ratio > height_ratio: ret = __resize_by_width(image, target_width)
    else: ret = __resize_by_height(image, target_height)
    height, width, _ = ret.shape
    if height < target_height or width < target_width: ret = __extend_to_exact_size(ret, target_width, target_height, default_background_color, refiner_on)
    return ret


def __resize_by_width(img, target_width):
    current_height, current_width, _ = img.shape
    target_height = int(target_width / current_width * current_height)
    ret = cv.resize(img, (target_width, target_height), interpolation=cv.INTER_LINEAR)
    return ret


def __resize_by_height(img, target_height):
    current_height, current_width, _ = img.shape
    target_width = int(target_height / current_height * current_width)
    ret = cv.resize(img, (target_width, target_height), interpolation=cv.INTER_LINEAR)
    return ret


def __extend_to_exact_size(img, target_width, target_height, default_background_color, refiner_on):
    current_height, current_width, _ = img.shape
    if refiner_on:
        left_border = 0
        top_border  = 0
    else:
        left_border = int((target_width - current_width) / 2)
        top_border = int((target_height - current_height) / 2)
        
    right_border = target_width - current_width - left_border
    bottom_border = target_height - current_height - top_border
    ret = cv.copyMakeBorder(img, top_border, bottom_border, left_border, right_border,cv.BORDER_CONSTANT, None, default_background_color)
    return ret

In [11]:
out_file    = open(path_out_file, "w")
annotations = os.listdir(path_labels)

for i in range (0, len(annotations)):#here we browse all videos
    video_name = annotations[i].split('/')[-1].split('\\')[-1].split('.')[0]
    data       = json.load(open(os.path.join(path_labels, annotations[i])))
    print("\n",video_name)
    
    ids_counter = np.zeros((2, 100000), dtype=int)
    
    for v in range (0,600): #here we browse all frames. Single movie has 600 frames
        img_name     = path_images+'/'+video_name+'/'+str(v)+".jpg"
        img = cv.imread(img_name)
        
        
        labels       = data['sequence'][v]
        str_to_write = img_name
        for c in range (0, len(classes)):
            try:
                for inst in data['sequence'][v][classes[c]]:
                    box           = inst['box2d']
                    if ((box[2]-box[0])*(box[3]-box[1])) < 900: #we do not want small boxes
                        continue
                           
                    act_id        = inst['id']
                    id_counter     = ids_counter[c,act_id]
                    ids_counter[c,act_id] += 1
                    crop = normalize_image_size(img[box[1]:box[3], box[0]:box[2], ...], 224, 224)
                    cv.imwrite(out_dir+'/'+str(c)+'_'+str(i)+'_'+str(act_id)+'_'+str(id_counter)+'_.jpg', crop)
                    out_file.write(out_dir+'/'+str(c)+'_'+str(i)+'_'+str(act_id)+'_'+str(id_counter)+'_.jpg\n')
                    print("\r  v={}, c={} :".format(v,c) +out_dir+'/'+str(c)+'_'+str(i)+'_'+str(act_id)+'_'+str(id_counter)+'_.jpg       ', end="")            
                          
            except Exception as e:
                #print("\n ★★★★★  except :{}".format(e))
                continue #nothing, the class is just not presented in the frame
out_file.close()
    
        


 train_00
  v=599, c=1 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/1_0_9510_0_.jpg         
 train_01
  v=599, c=1 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/1_1_385_146_.jpg       
 train_02
  v=599, c=1 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/1_2_4498_3_.jpg        
 train_03
  v=599, c=0 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/0_3_3531_8_.jpg         
 train_04
  v=599, c=1 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/1_4_37465_4_.jpg        
 train_05
  v=599, c=1 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/1_5_35238_6_.jpg        
 train_06
  v=599, c=0 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/0_6_5336_158_.jpg       
 train_07
  v=599, c=1 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/1_7_37925_11_.jpg        
 train_08
  v=599, c=1 :D:\SIGNATE\Signate_3rd_AI_edge_competition\train_imgs_matcher/1_8_7453_4_.jpg        
 t